In [11]:
from ipynb.fs.full.utils import read_test_input
from ipynb.fs.full.opcode import Opcoder
from ipynb.fs.full.amplifiers import run_sequence, run_feedback_sequence

from itertools import permutations

In [12]:
filename = "day7_input.txt"

In [13]:
read_input = read_test_input(filename)

In [14]:
read_input = read_input[0].split(',')

In [15]:
read_input = [int(val) for val in read_input]

In [16]:
program = read_input

In [17]:
#Part1
max(run_sequence(program, sequence) for sequence in permutations([0,1,2,3,4], 5))

212460

In [18]:
#Part2
max(run_feedback_sequence(program, sequence) for sequence in permutations([5,6,7,8,9], 5))

21844737